In [12]:
import os
import pandas as pd
import matplotlib as mpl
import matplotlib_inline
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.spatial.ckdtree import coo_entries

DATA = 'data'

In [13]:
vegetable_oil_production = pd.read_csv(os.path.join(DATA, 'vegetable-oil-production.csv'))  # need to change the code to use FAO data
vegetable_oil_production = vegetable_oil_production[vegetable_oil_production['Entity'] != 'World']
year = vegetable_oil_production['Year'].drop_duplicates(keep='first', inplace=False)
vegetable_oil_production.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11010 entries, 0 to 11063
Data columns (total 16 columns):
 #   Column                                                                     Non-Null Count  Dtype  
---  ------                                                                     --------------  -----  
 0   Entity                                                                     11010 non-null  object 
 1   Code                                                                       9269 non-null   object 
 2   Year                                                                       11010 non-null  int64  
 3   Crops processed - Oil, soybean - 237 - Production - 5510 - tonnes          5685 non-null   float64
 4   Crops processed - Oil, sesame - 290 - Production - 5510 - tonnes           4158 non-null   float64
 5   Crops processed - Oil, linseed - 334 - Production - 5510 - tonnes          4333 non-null   float64
 6   Crops processed - Oil, palm - 257 - Production - 5510 

In [14]:
veg_oil_by_country = vegetable_oil_production[~vegetable_oil_production['Code'].isnull()].copy()
veg_oil_by_country

,Entity,Code,Year,"Crops processed - Oil, soybean - 237 - Production - 5510 - tonnes","Crops processed - Oil, sesame - 290 - Production - 5510 - tonnes","Crops processed - Oil, linseed - 334 - Production - 5510 - tonnes","Crops processed - Oil, palm - 257 - Production - 5510 - tonnes","Crops processed - Oil, rapeseed - 271 - Production - 5510 - tonnes","Crops processed - Oil, groundnut - 244 - Production - 5510 - tonnes","Crops processed - Oil, cottonseed - 331 - Production - 5510 - tonnes","Crops processed - Oil, coconut (copra) - 252 - Production - 5510 - tonnes","Crops processed - Oil, olive, virgin - 261 - Production - 5510 - tonnes","Crops processed - Oil, safflower - 281 - Production - 5510 - tonnes","Crops processed - Oil, sunflower - 268 - Production - 5510 - tonnes","Crops processed - Oil, maize - 60 - Production - 5510 - tonnes","Crops processed - Oil, palm kernel - 258 - Production - 5510 - tonnes"
0,Afghanistan,AFG,1961,NaN,2253.0,3531.0,NaN,NaN,NaN,4997.0,NaN,82.0,NaN,2938.0,NaN,NaN
1,Afghanistan,AFG,1962,NaN,1876.0,3701.0,NaN,NaN,NaN,7716.0,NaN,90.0,NaN,3138.0,NaN,NaN
2,Afghanistan,AFG,1963,NaN,1831.0,2857.0,NaN,NaN,NaN,11742.0,NaN,82.0,NaN,3138.0,NaN,NaN
3,Afghanistan,AFG,1964,NaN,2722.0,3377.0,NaN,NaN,NaN,7960.0,NaN,90.0,NaN,3138.0,NaN,NaN
4,Afghanistan,AFG,1965,NaN,2821.0,4327.0,NaN,NaN,NaN,7926.0,NaN,82.0,NaN,3238.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11059,Zimbabwe,ZWE,1969,NaN,NaN,NaN,NaN,NaN,4242.0,3004.0,NaN,NaN,NaN,333.0,1800.0,NaN
11060,Zimbabwe,ZWE,1970,NaN,NaN,NaN,NaN,NaN,6006.0,9180.0,NaN,NaN,NaN,832.0,2250.0,NaN
11061,Zimbabwe,ZWE,1971,NaN,NaN,NaN,NaN,NaN,4284.0,10926.0,NaN,NaN,NaN,666.0,3600.0,NaN
11062,Zimbabwe,ZWE,1972,NaN,NaN,NaN,NaN,NaN,7980.0,11844.0,NaN,NaN,NaN,999.0,4500.0,NaN


In [15]:
veg_oil_by_area = vegetable_oil_production[vegetable_oil_production['Code'].isnull()].copy()
veg_oil_by_area

,Entity,Code,Year,"Crops processed - Oil, soybean - 237 - Production - 5510 - tonnes","Crops processed - Oil, sesame - 290 - Production - 5510 - tonnes","Crops processed - Oil, linseed - 334 - Production - 5510 - tonnes","Crops processed - Oil, palm - 257 - Production - 5510 - tonnes","Crops processed - Oil, rapeseed - 271 - Production - 5510 - tonnes","Crops processed - Oil, groundnut - 244 - Production - 5510 - tonnes","Crops processed - Oil, cottonseed - 331 - Production - 5510 - tonnes","Crops processed - Oil, coconut (copra) - 252 - Production - 5510 - tonnes","Crops processed - Oil, olive, virgin - 261 - Production - 5510 - tonnes","Crops processed - Oil, safflower - 281 - Production - 5510 - tonnes","Crops processed - Oil, sunflower - 268 - Production - 5510 - tonnes","Crops processed - Oil, maize - 60 - Production - 5510 - tonnes","Crops processed - Oil, palm kernel - 258 - Production - 5510 - tonnes"
54,Africa,NaN,1961,2337.0,23706.0,10917.0,1131882.0,31180.0,448152.0,166846.0,35923.0,77143.0,3810.0,33811.0,60192.0,77279.0
55,Africa,NaN,1962,2893.0,30158.0,9627.0,1111006.0,22141.0,517235.0,163605.0,37781.0,87970.0,3752.0,30479.0,62217.0,72627.0
56,Africa,NaN,1963,853.0,30177.0,7144.0,1145004.0,28708.0,577561.0,211456.0,44069.0,143734.0,3818.0,33889.0,64525.0,67875.0
57,Africa,NaN,1964,2312.0,25252.0,10791.0,1160831.0,26614.0,552984.0,180852.0,42435.0,143327.0,3703.0,30011.0,62547.0,78443.0
58,Africa,NaN,1965,1984.0,29208.0,14285.0,1138860.0,24215.0,646906.0,242830.0,42180.0,112610.0,3784.0,27358.0,65229.0,95883.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10812,Western Asia,NaN,2010,528876.0,92769.0,2270.0,NaN,478099.0,11589.0,286082.0,7.0,443892.0,0.0,744693.0,68699.0,NaN
10813,Western Asia,NaN,2011,455841.0,89268.0,12924.0,NaN,446379.0,10886.0,332377.0,227.0,461490.0,0.0,838192.0,64937.0,NaN
10814,Western Asia,NaN,2012,395594.0,91205.0,34718.0,NaN,378278.0,9998.0,308910.0,133.0,471316.0,37.0,795581.0,82235.0,NaN
10815,Western Asia,NaN,2013,391071.0,82693.0,16731.0,NaN,369019.0,9577.0,280947.0,27.0,416862.0,42.0,837072.0,98291.0,NaN


In [16]:
veg_oil_yearly_production = veg_oil_by_country.groupby('Year').sum()
veg_oil_yearly_production.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54 entries, 1961 to 2014
Data columns (total 13 columns):
 #   Column                                                                     Non-Null Count  Dtype  
---  ------                                                                     --------------  -----  
 0   Crops processed - Oil, soybean - 237 - Production - 5510 - tonnes          54 non-null     float64
 1   Crops processed - Oil, sesame - 290 - Production - 5510 - tonnes           54 non-null     float64
 2   Crops processed - Oil, linseed - 334 - Production - 5510 - tonnes          54 non-null     float64
 3   Crops processed - Oil, palm - 257 - Production - 5510 - tonnes             54 non-null     float64
 4   Crops processed - Oil, rapeseed - 271 - Production - 5510 - tonnes         54 non-null     float64
 5   Crops processed - Oil, groundnut - 244 - Production - 5510 - tonnes        54 non-null     float64
 6   Crops processed - Oil, cottonseed - 331 - Production - 

In [17]:
import re  # wilson you're obviously using regext ot update the column headers. please explain this one to me
pattern = r'(?<=Oil, ).+?(?= - \d)'
cols = [re.search(pattern, c, re.RegexFlag.IGNORECASE)[0] for c in veg_oil_yearly_production]
cols = [re.sub(' ', '_', c) for c in cols]
cols = [re.sub('\W', '', c) for c in cols]

In [18]:
veg_oil_yearly_production.columns = cols
veg_oil_yearly_production.reset_index(inplace=True)
veg_oil_yearly_production.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Year           54 non-null     int64  
 1   soybean        54 non-null     float64
 2   sesame         54 non-null     float64
 3   linseed        54 non-null     float64
 4   palm           54 non-null     float64
 5   rapeseed       54 non-null     float64
 6   groundnut      54 non-null     float64
 7   cottonseed     54 non-null     float64
 8   coconut_copra  54 non-null     float64
 9   olive_virgin   54 non-null     float64
 10  safflower      54 non-null     float64
 11  sunflower      54 non-null     float64
 12  maize          54 non-null     float64
 13  palm_kernel    54 non-null     float64
dtypes: float64(13), int64(1)
memory usage: 6.0 KB


In [19]:
veg_oil_fig = px.area(
    veg_oil_yearly_production,
    x='Year',
    y=veg_oil_yearly_production.columns[1:]
)
veg_oil_fig.update_traces(textfont_size=16, hovertemplate=None)
veg_oil_fig.update_layout(hovermode="x")

veg_oil_fig.show()